# Train cNN network with fonts png

## imports

In [1]:
from __future__ import absolute_import, division, print_function, unicode_literals
from tensorflow.keras import layers
from tensorflow import keras

import numpy as np
import tensorflow as tf
import PIL

/home/andrew/.local/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/home/andrew/.local/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/home/andrew/.local/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/home/andrew/.local/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:519: 

## load dataset from npz

In [2]:
path = '/home/andrew/Recognizerka/fonts/png32_rotated.npz'
with np.load(path) as data:
    x_train = data['x_train']
    y_train = data['y_train']
    x_test = data['x_test']
    y_test = data['y_test']

In [3]:
# Функция превращения изобращения в валидный для предсказания np.array
def getArrayFromImage(path):
  img = Image.open(path).convert('L')
  arr = np.array(img).reshape(32,32,1)
  return arr

In [19]:
RUS_CHARS = [
    'А'.encode('utf-8'),
    'Б'.encode('utf-8'),
    'В'.encode('utf-8'),
    'Г'.encode('utf-8'),
    'Д'.encode('utf-8'),
    'Е'.encode('utf-8'),
    'Ж'.encode('utf-8'),
    'З'.encode('utf-8'),
    'И'.encode('utf-8'),
    'Й'.encode('utf-8'),
    'К'.encode('utf-8'),
    'Л'.encode('utf-8'),
    'М'.encode('utf-8'),
    'Н'.encode('utf-8'),
    'О'.encode('utf-8'),
    'П'.encode('utf-8'),
    'Р'.encode('utf-8'),
    'С'.encode('utf-8'),
    'Т'.encode('utf-8'),
    'У'.encode('utf-8'),
    'Ф'.encode('utf-8'),
    'Х'.encode('utf-8'),
    'Ц'.encode('utf-8'),
    'Ч'.encode('utf-8'),
    'Ш'.encode('utf-8'),
    'Щ'.encode('utf-8'),
    'Ъ'.encode('utf-8'),
    'Ы'.encode('utf-8'),
    'Ь'.encode('utf-8'),
    'Э'.encode('utf-8'),
    'Ю'.encode('utf-8'),
    'Я'.encode('utf-8'),
    'Ё'.encode('utf-8'),
    'а'.encode('utf-8'),
    'б'.encode('utf-8'),
    'в'.encode('utf-8'),
    'г'.encode('utf-8'),
    'д'.encode('utf-8'),
    'е'.encode('utf-8'),
    'ж'.encode('utf-8'),
    'з'.encode('utf-8'),
    'и'.encode('utf-8'),
    'й'.encode('utf-8'),
    'к'.encode('utf-8'),
    'л'.encode('utf-8'),
    'м'.encode('utf-8'),
    'н'.encode('utf-8'),
    'о'.encode('utf-8'),
    'п'.encode('utf-8'),
    'р'.encode('utf-8'),
    'с'.encode('utf-8'),
    'т'.encode('utf-8'),
    'у'.encode('utf-8'),
    'ф'.encode('utf-8'),
    'х'.encode('utf-8'),
    'ц'.encode('utf-8'),
    'ч'.encode('utf-8'),
    'ш'.encode('utf-8'),
    'щ'.encode('utf-8'),
    'ъ'.encode('utf-8'),
    'ы'.encode('utf-8'),
    'ь'.encode('utf-8'),
    'э'.encode('utf-8'),
    'ю'.encode('utf-8'),
    'я'.encode('utf-8'),
    'ё'.encode('utf-8'),
]

In [35]:
from PIL import Image
from random import *

ran = randint(0, x_train.shape[0])
print("Random: ", ran)

a = x_train[ran]
a = a.reshape(32, 32)
img = Image.fromarray(a)
img.show()
print("Char: ", RUS_CHARS[int(y_train[ran])].decode('utf-8'))

Random:  1270
Char:  г


## configuration of cNN

In [28]:
model = tf.keras.models.Sequential([    
    tf.keras.layers.Conv2D(32, (6, 6), activation='relu', input_shape=(32, 32, 1)),
    tf.keras.layers.MaxPooling2D(pool_size=(2, 2)),
    
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(256, activation='relu'),
    tf.keras.layers.Dense(66, activation='softmax')
])


In [31]:
# Подготовка данных для запуска нейронной сети 
x_train = x_train.astype('float32') / 255
x_test = x_test.astype('float32') / 255

y_train = y_train.astype('float32')
y_test = y_test.astype('float32')

# Зарезервируем 1000 элементов для валидации
x_val = x_train[-1000:]
y_val = y_train[-1000:]
x_train = x_train[:-1000]
y_train = y_train[:-1000]

model.compile(optimizer=keras.optimizers.RMSprop(),  # Optimizer
              # Loss function to minimize
              loss=keras.losses.SparseCategoricalCrossentropy(
                  from_logits=True),
              # List of metrics to monitor
              metrics=['accuracy'])

In [32]:
print('# Fit model on training data')
history = model.fit(x_train, y_train,
                    batch_size=64,
                    epochs=30,
                    shuffle=True,
                    validation_data=(x_val, y_val))

# Fit model on training data
Train on 31270 samples, validate on 1000 samples
Epoch 1/30
31270/31270 [==============================] - 17s 530us/sample - loss: 4.1897 - acc: 0.0154 - val_loss: 4.1898 - val_acc: 0.0000e+00
Epoch 2/30
31270/31270 [==============================] - 16s 517us/sample - loss: 4.1896 - acc: 0.0156 - val_loss: 4.1901 - val_acc: 0.0000e+00
Epoch 3/30
31270/31270 [==============================] - 16s 526us/sample - loss: 4.1896 - acc: 0.0146 - val_loss: 4.1903 - val_acc: 0.0000e+00
Epoch 4/30
 2368/31270 [=>............................] - ETA: 15s - loss: 4.1896 - acc: 0.0165

KeyboardInterrupt: 

In [ ]:
print('\nhistory dict:', history.history)

In [ ]:
# Evaluate the model on the test data using `evaluate`
print('\n# Evaluate on test data')
results = model.evaluate(x_test, y_test, batch_size=64)
print('test loss, test acc:', results)

In [ ]:
# create seporate test set
files = [
    '18847_l_6.png',
    '19046_u_9.png',
    '19051_l_0.png',
    '19081_l_0.png',
    '19081_u_17.png'
]
path_to_files = "/home/andrew/Recognizerka/fonts/png_32_32/"

flen = len(files)
my_test_x = np.zeros( (flen, 32, 32, 1) )
print(my_test_x.shape)
k = 0
for file in files:
    my_test_x[k] = getFlatArrayFromImage(path_to_files+file).reshape(32, 32, 1)
    k+=1

# Generate predictions (probabilities -- the output of the last layer)
# on new data using `predict`
print('\n# Generate predictions for 3 samples')
predictions = model.predict(my_test_x)
print('predictions shape:', predictions.shape)

In [ ]:
def get_char_from_pred(prediction):
    from draft import RUS_CHARS
    return RUS_CHARS[np.argmax(prediction)].decode('utf-8')

for pred in predictions:
    print(get_char_from_pred(pred))